In [1]:
import io
import numpy
from PIL import Image
import aiohttp
import asyncio

In [7]:
base_url = "https://wxs.ign.fr/an7nvfzojv5wa96dsga5nk8w/geoportail/wmts"
_headers = {
        'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'Host': 'wxs.ign.fr',
        'Referer': 'http://www.geoportail.gouv.fr/swf/geoportal-visu-1.3.2.swf',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:22.0) Gecko/20100101 Firefox/22.0',
}
default_values = {
    'layer' : 'GEOGRAPHICALGRIDSYSTEMS.MAPS.SCAN-EXPRESS.STANDARD',
    'style' : 'normal',
    'tilematrixset' : 'PM',
    'Service' : 'WMTS',
    'Request' : 'GetTile',
    'Version' : '1.0.0',
    'Format': 'image/jpeg',
    'TileMatrix' : 16
    }
test_values = {
    'layer' : 'GEOGRAPHICALGRIDSYSTEMS.MAPS.SCAN-EXPRESS.STANDARD',
    'style' : 'normal',
    'tilematrixset' : 'PM',
    'Service' : 'WMTS',
    'Request' : 'GetFeatureInfo',
    'Version' : '1.0.0',
    'Format': 'image/jpeg',
    'TileMatrix' : 16,
    'InfoFormat' : 'text/html',
    'i' : 1,
    'j' : 1
    }


In [10]:
def get_map(start_row, start_col, row_number, col_number):
    
    contenant = numpy.empty([row_number,col_number,256,256,3], dtype = 'uint8')
    
    async def put_tile(session, Row, Col):
        data = await get_tile(session, start_row+Row, start_col+Col)
        contenant[Row][Col] = Image.open(io.BytesIO(data))
        
    async def get_tile(session, Row, Col):
        _values = default_values
        t_values = test_values
        t_values['TileCol'] = _values['TileCol'] = Col
        t_values['TileRow'] = _values['TileRow'] = Row
        #async with session.request('GET', base_url, params = t_values) as resp:
        #    assert resp.status == 200
        #    print(resp.read())
        async with session.request('GET', base_url, params = _values) as resp:
            assert resp.status == 200
            return await  resp.read()
        
    async def main(loop):
        tasks = []
        async with aiohttp.ClientSession(loop=loop, headers = _headers ) as session:
            for j in range(0,col_number):
                for i in range(0,row_number):
                    task = asyncio.ensure_future(put_tile(session, i, j))
                    tasks.append(task)
            await asyncio.gather(*tasks)
            

    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(main(loop))
    loop.run_until_complete(future)
    
    return contenant

In [17]:
test = get_map(23961,33036,20,20)

In [18]:
Image.fromarray(numpy.concatenate(numpy.concatenate(test, axis= 1 ), axis=1)).save("output.jpg")